# Getting Started with Ollama: Local LLMs for GenAI Apps

> **Note:** You need Ollama installed on your machine and its server running on `http://localhost:11434`.

### 1- What is Ollama 
- Ollama is a lightweight runtime that serves open‑source LLMs locally (CPU or GPU) via a simple HTTP API on `localhost:11434`.
- visit https://ollama.com/search to check all the available models
#### a- Install Ollama
**Download**: check this page and download the version for your OS: https://ollama.com/download

After installation, start the server:

```bash
ollama serve
```

Check version:
```bash
ollama --version
```

To bind on a custom host/port:
```bash
OLLAMA_HOST=0.0.0.0:11434 ollama serve
```

#### b- Pull a model
Run this **in a terminal** (first run downloads weights):

```bash
ollama pull llama3.1
```

You can also try `mistral`, `qwen2.5`, or a vision model like `llama3.2-vision`. Models available in https://ollama.com/search
#### c- Check and run the model in terminal
Run this **in a terminal**:

```bash
ollama list
```
the if the model is downloaded:

```bash
ollama run llama3.1
```


### 2) Verify the server is reachable
We ping the local API and display available models.

In [ ]:
import json, requests
from pprint import pprint

BASE_URL = 'http://localhost:11434'

def check_server(base_url=BASE_URL):
    try:
        r = requests.get(base_url)
        return r.status_code, r.text[:200]
    except Exception as e:
        return None, str(e)

status, info = check_server()
print('Server status:', status)
print('Info (truncated):', info)

def list_models(base_url=BASE_URL):
    try:
        r = requests.get(base_url + '/api/tags', timeout=10)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        return {'error': str(e)}

pprint(list_models())


### 3) Python client install
Install the official Python package that talks to the local Ollama API.

In [ ]:
# If needed, uncomment and run:
# !pip install ollama

#### Simple chat completion (non‑streaming)
This uses the `ollama` client. 

In [ ]:
try:
    from ollama import Client
    client = Client(host=BASE_URL)
    resp = client.chat(
        model='llama3.2:1b',
        messages=[
            {'role': 'system', 'content': 'You are a concise teaching assistant.'},
            {'role': 'user', 'content': 'Explain GANs in 4 bullet points.'}
        ],
        options={'temperature': 0.7, 'num_ctx': 4096}
    )
    print(resp['message']['content'])
except Exception as e:
    print('⚠️ Chat call failed. Ensure the server is running and the model is pulled. Error:\n', e)


#### Streaming tokens (nice for demos)
The server can stream partial responses. This is useful in apps to show incremental output.

In [ ]:
try:
    from ollama import Client
    client = Client(host=BASE_URL)
    stream = client.chat(
        model='llama3.2:1b',
        messages=[{'role': 'user', 'content': 'List 5 use cases of RAG.'}],
        stream=True,
    )
    for chunk in stream:
        print(chunk['message']['content'], end='', flush=True)
    print()
except Exception as e:
    print('⚠️ Streaming call failed. Error:\n', e)


#### Embeddings for mini‑RAG
We generate an embedding vector for a short text. You can index multiple texts and do cosine similarity to retrieve relevant passages.

In [ ]:
try:
    from ollama import Client
    import numpy as np
    client = Client(host=BASE_URL)
    text = 'Diffusion models iteratively denoise data to sample from complex distributions.'
    emb = client.embeddings(model='llama3.2:1b', prompt=text)
    vec = np.array(emb['embedding'])
    print('Embedding length:', len(vec))
    print('First 8 dims:', vec[:8])
except Exception as e:
    print('⚠️ Embedding call failed. Error:\n', e)


#### Raw REST with `requests` (non‑streaming)
Sometimes you want to see the plain HTTP calls for debugging or to avoid extra dependencies.

In [ ]:
import json, requests
payload = {
  'model': 'llama3.2:1b',
  'messages': [
    {'role': 'system', 'content': 'Be brief.'},
    {'role': 'user', 'content': 'What is a VAE, in 3 sentences max?'}
  ],
  'options': {'temperature': 0.6},
  'stream': False
}
try:
    r = requests.post(BASE_URL + '/api/chat', json=payload, timeout=120)
    r.raise_for_status()
    print(r.json()['message']['content'])
except Exception as e:
    print('⚠️ REST call failed. Error:\n', e)
